In [1]:
import numpy as np
import pandas as pd

There are 54 labels in the default scheme

it generates values for the whole head (generation_labels),
but only retain segmentations related to the brain (output_labels)

CSF(24) is not retained

In [2]:
# the output_labels are the labels that synthseg generates at test time
output_labels = np.load('../../data/labels_classes_priors/synthseg_segmentation_labels.npy')
generation_labels = np.load('../../data/labels_classes_priors/generation_labels.npy')
generation_classes = np.load('../../data/labels_classes_priors/generation_classes_contrast_specific.npy')
classes_names = np.load('../../data/labels_classes_priors/synthseg_segmentation_names.npy')
df = pd.DataFrame(np.vstack((classes_names,generation_classes, generation_labels, output_labels)).T)
df.columns = ['classes_names', 'generation_classes', 'generation_labels', 'output_labels']


In [3]:
df

,classes_names,generation_classes,generation_labels,output_labels
0,background,0,0,0
1,3rd ventricle,1,14,14
2,4th ventricle,1,15,15
3,brain-stem,13,16,16
4,background,1,24,0
5,background,1,72,0
6,background,2,85,0
7,background,14,502,0
8,background,15,506,0
9,background,16,507,0


In [4]:
# Create a copy of df
df_filtered = df.copy()

# Change name to 'CSF' where generation_label is 24
df_filtered.loc[df_filtered['generation_labels'] == '24', 'classes_names'] = 'CSF'
df_filtered.loc[df_filtered['generation_labels'] == '24', 'output_labels'] = '24'


# Filter out rows where class name is 'background', except for CSF
df_filtered = df_filtered[
    (df_filtered['classes_names'] != 'background') ]
df_filtered
# Remove duplicate class names, keeping first occurrence
df_filtered = df_filtered.drop_duplicates(subset=['classes_names'], keep='first')
# Sort the DataFrame by output_labels column, converting to numeric for proper sorting
df_filtered = df_filtered.astype({'output_labels': int}).sort_values('output_labels')

# Reset the index after filtering
df_filtered = df_filtered.reset_index(drop=True)

# Save the filtered DataFrame to CSV without the index
df_filtered.to_csv('filtered_labels.csv', index=False)

df_filtered


,classes_names,generation_classes,generation_labels,output_labels
0,left cerebral white matter,2,2,2
1,left cerebral cortex,3,3,3
2,left lateral ventricle,1,4,4
3,left inferior lateral ventricle,1,5,5
4,left cerebellum white matter,4,7,7
5,left cerebellum cortex,5,8,8
6,left thalamus,6,10,10
7,left caudate,3,11,11
8,left putamen,7,12,12
9,left pallidum,8,13,13


In [5]:
# a lot of values are treated as background (0) in the output_labels
np.unique(output_labels) 

array([ 0,  2,  3,  4,  5,  7,  8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 26,
       28, 41, 42, 43, 44, 46, 47, 49, 50, 51, 52, 53, 54, 58, 60],
      dtype=int32)

In [7]:
prior_means = np.load('../../data/labels_classes_priors/prior_means_t1.npy')
prior_stds = np.load('../../data/labels_classes_priors/prior_stds_t1.npy')

print(prior_means[:,4])
print(prior_means[:,14])

print(prior_means)
print(prior_stds)


[135.95211069  13.41123838]
[126.37696696  24.79729954]
[[  7.          44.43852646 160.29378443  96.29526152 135.95211069
   88.15506261 129.52385575 124.05468047 151.79945792 142.53280645
  116.10473738  86.83433108  59.62853626 129.78496267 126.37696696
   18.11697145  93.90532281 117.13488728  69.91289748 113.7338777
   68.23628889  48.42668289  30.19849003  94.24268292 144.4557237 ]
 [  1.75        11.21000137  21.65584467  12.17831661  13.41123838
   10.55970441  17.68538813  15.72768241  19.51073198  16.05725779
   14.8217615   17.47509896  10.4086569   12.2351825   24.79729954
    3.8432211    9.12899967  11.66694586   9.24940558  12.41801427
   11.09552493   8.56551669   6.86072804  18.31996116  16.31849085]]
[[ 5.         25.56555604 15.63223578 18.91828816 15.34013798 17.0927207
  19.58860262 13.80592264  8.55665307 21.12773275 19.45680845 17.16671351
  23.65403494 17.53320268 83.95184226 13.85991432 48.60145092 64.79087693
  25.46535693 74.86602865 35.85704951 19.91995259 2

In [30]:
# test synthseg segmentation results
import nibabel as nib
# Load a test segmentation file
seg_img = nib.load(r'/mnt/hdd0/download/oasis-1/brain_segmentations/OAS1_0001_synthseg.nii.gz')
seg_data = seg_img.get_fdata()

# Get unique values and print them
unique_vals = np.unique(seg_data)
print("Unique segmentation values:")
print(unique_vals)


Unique segmentation values:
[ 0.  2.  3.  4.  5.  7.  8. 10. 11. 12. 13. 14. 15. 16. 17. 18. 24. 26.
 28. 41. 42. 43. 44. 46. 47. 49. 50. 51. 52. 53. 54. 58. 60.]


In [31]:
len(unique_vals)

33